In [1]:
import os
import pandas as pd
import numpy as np
# import validation_data_input
import plotly.express as px
import toml
import polars as pl
from pathlib import Path
import util
import psrc_theme

# to show plotly figures in quarto HTML file
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"
pio.templates.default = "simple_white+psrc_color" # set plotly template

In [3]:
# %store -r validation_data

# config = validation_data.config.copy()
# tour = validation_data.tour.copy()
# person = validation_data.person.copy()

config = toml.load(Path(Path.cwd(), '..\..\..\..\configuration', 'validation_configuration.toml'))
input_config = toml.load(Path(Path.cwd(), '..\..\..\..\configuration', 'input_configuration.toml'))

data = util.ValidationData(config,input_config,['person', 'tour'])

person = data.person.to_pandas()
tour = data.tour.to_pandas()

In [4]:
df_tour = tour.copy()
df_person = person.copy()

# add person type labels
ptype_cat = {1: "1: full time worker",
             2: "2: part time worker",
             3: "3: non-worker age 65+",
             4: "4: other non-working adult",
             5: "5: university student",
             6: "6: grade school student/child age 16+",
             7: "7: child age 5-15",
             8: "8: child age 0-4"}
df_person['pptyp_label'] = df_person['pptyp'].map(ptype_cat)

pdpurp_cat = {1: "1: Work",
              2: "2: School",
              3: "3: Escort",
              4: "4: Personal Business",
              5: "5: Shop",
              6: "6: Meal",
              7: "7: Social",
              8: "8: Social",
              9: "9: Personal Business"}
df_tour['pdpurp_label'] = df_tour['pdpurp'].map(pdpurp_cat)
# df_tour['pdpurp_label'] = df_tour['pdpurp_label'].astype(pd.CategoricalDtype(["Work","School","Shop","Meal","Escort","Personal Business"]))

df_tour = df_tour.merge(df_person, how='left', on=['pno','hhno','source'])

In [5]:
wk_base_subtour = df_tour.loc[(df_tour['parent']!=0)].copy()
wk_tour = df_tour.loc[(df_tour['parent']==0) & (df_tour['pdpurp']==1)].copy()

In [6]:
test = wk_tour.loc[wk_tour['subtrs']!=11].copy()
df_plot = test.groupby(['source','subtrs']).agg(
    total=pd.NamedAgg(column="toexpfac", aggfunc="sum"),
    sample_size=pd.NamedAgg(column="toexpfac", aggfunc="size")).reset_index()

df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['total']. \
    apply(lambda x: x / float(x.sum()))


fig = px.bar(df_plot.sort_values(by=['source']), x="subtrs", y="percentage", color="source",
             barmode="group",hover_data=['total','sample_size'],title="number of workbased subtours in work tours")
fig.update_layout(height=400, width=700, font=dict(size=11),
                  xaxis = dict(dtick = 1, categoryorder='category ascending'),
                  yaxis=dict(tickformat=".2%"))
fig.show()

## workbased subtour by purpose

In [7]:
df_plot = wk_base_subtour.groupby(['source','pdpurp_label']).agg(
    total=pd.NamedAgg(column="toexpfac", aggfunc="sum"),
    sample_size=pd.NamedAgg(column="toexpfac", aggfunc="size")).reset_index()
df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['total']. \
    apply(lambda x: x / float(x.sum()))


fig = px.bar(df_plot.sort_values(by=['source']), x="pdpurp_label", y="percentage", color="source",
             barmode="group",hover_data=['total','sample_size'],title="workbased subtour by purpose")
fig.update_layout(height=400, width=700, font=dict(size=11),
                  yaxis=dict(tickformat=".2%"))
fig.show()

## workbased subtour by person type

In [8]:
df_plot = wk_base_subtour.groupby(['source','pptyp_label']).agg(
    total=pd.NamedAgg(column="toexpfac", aggfunc="sum"),
    sample_size=pd.NamedAgg(column="toexpfac", aggfunc="size")).reset_index()
df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['total']. \
    apply(lambda x: x / float(x.sum()))


fig = px.bar(df_plot.sort_values(by=['source']), x="pptyp_label", y="percentage", color="source",
             barmode="group",hover_data=['total','sample_size'],title="workbased subtour by person type")
fig.update_layout(height=400, width=700, font=dict(size=11),
                  yaxis=dict(tickformat=".2%"))
fig.show()